Prototype Notebook
Kevin McCoy

In [ ]:
# For any questions regarding this notebook, please contact Kevin McCoy <kmccoy8@gatech.edu>

In [ ]:
import smtplib
from string import Template
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
MY_ADDRESS = 'emailnotif189382@gmail.com'#'email.notif1403@gmail.com'
PASSWORD = 'MitchLin1'#'JacketsRule12343'
DESTINATION = 'kmccoy8@gatech.edu'

In [ ]:
def send_notif(subject, msg_text):
    # set up the SMTP server
    s = smtplib.SMTP(host='smtp.gmail.com', port=587)
    s.starttls()
    s.login(MY_ADDRESS, PASSWORD)
    # For each contact, send the email:
    msg = MIMEMultipart()       # create a message
    # add in the actual person name to the message template
    # setup the parameters of the message
    msg['From']=MY_ADDRESS
    msg['To']=DESTINATION
    msg['Subject']=subject
    # add in the message body
    msg.attach(MIMEText(msg_text, 'plain'))
    # send the message via the server set up earlier.
    s.send_message(msg)
    del msg
    # Terminate the SMTP session and close the connection
    s.quit()

In [ ]:
send_notif("Success Source Definition", "hello world")

In [ ]:
from semnet2 import offline, offline_hetesim, randomized_hetesim
import pandas as pd

In [ ]:
# Load in semnet database (should take no more than 20 seconds)
semnet_df = pd.read_csv("edges_updated.csv", index_col=0)


In [ ]:
semnet_df.head()

In [ ]:
# Lookup CUIs of interest

In [ ]:
import lxml.html as lh
from lxml.html import fromstring
import requests
import json

uri="https://utslogin.nlm.nih.gov"
auth_endpoint = "/cas/v1/api-key"

class Authentication:
    
    def __init__(self, apikey):
        self.apikey=apikey
        self.service="http://umlsks.nlm.nih.gov"

    def gettgt(self):
        params = {'apikey': self.apikey}
        h = {"Content-type": "application/x-www-form-urlencoded", "Accept": "text/plain", "User-Agent":"python" }
        r = requests.post(uri+auth_endpoint,data=params,headers=h)
        response = fromstring(r.text)
        ## extract the entire URL needed from the HTML form (action attribute) returned - looks similar to https://utslogin.nlm.nih.gov/cas/v1/tickets/TGT-36471-aYqNLN2rFIJPXKzxwdTNC5ZT7z3B3cTAKfSc5ndHQcUxeaDOLN-cas
        ## we make a POST call to this URL in the getst method
        tgt = response.xpath('//form/@action')[0]
        return tgt

    def getst(self,tgt):
        params = {'service': self.service}
        h = {"Content-type": "application/x-www-form-urlencoded", "Accept": "text/plain", "User-Agent":"python" }
        r = requests.post(tgt,data=params,headers=h)
        st = r.text
        return st
    

def get_cuis_for_concept(concept_name, page_size=1000, return_names=False):
    '''
    Get CUIs matching a query concept in UMLS
    '''
    API_KEY = '784e30e6-1938-47c6-8479-d1c9b654c704'
    VERSION = 'current'
    AuthClient = Authentication(API_KEY)

    tgt = AuthClient.gettgt()
    url = "https://uts-ws.nlm.nih.gov/rest/"

    content_endpoint = f'search/{VERSION}'

    query = {'ticket':AuthClient.getst(tgt), 'string':concept_name, 'pageSize':page_size}
    
    r = requests.get(url=url+content_endpoint, params=query)
    r.encoding = 'utf-8'
    items = json.loads(r.text)
    if return_names:
        data = [[res['ui'], res['name']] for res in items['result']['results']]
    else:
        data = [res['ui'] for res in items['result']['results']]
    return data




In [ ]:
get_cuis_for_concept("ejection fraction", return_names=True)[:5]

In [ ]:
toy_graph = pd.DataFrame([['a','t1','b','t1','r1',1],
                          ['a','t1','c','t2','r1',1],
                          ['c','t2','d','t1','r1',1],
                          ['b','t1','e','t2','r1',1],
                          ['a','t1','e','t2','r1',1],
                          ['c','t2','f','t2','r1',1],
                          ['d','t1','b','t1','r2',1],
                          ['e','t2','c','t2','r2',1],
                          ['a','t1','f','t2','r2',1],
                          ['f','t2','d','t1','r2',1],
                          ['b','t1','a','t1','r2',1],
                          ['a','t1','a','t1','r1',1]], 
                         columns=['start_node',
                                  'start_type',
                                  'end_node',
                                  'end_type',
                                  'relation',
                                  'weight'])

In [ ]:
# Do this (should take ~100s)
edgelist = semnet_df.to_dict(orient='records')


In [ ]:

import pickle 

infile = open("rel2inv.pkl",'rb')
rel2inv = pickle.load(infile)
infile.close()


In [ ]:
rel2inv

In [ ]:
# (takes ~s)
semnet_graph = offline.HetGraph(edgelist, rel2inv)

In [ ]:
toy_graph.get_max_one_sided_k()

In [ ]:
offline_hetesim.mean_hetesim_scores(toy_graph, ["s", "a"], "d", 2)

In [ ]:
output_fn = "output_file.csv"
df.to_csv(output_fn)